# VG American

In [1]:
import numpy as np

In [75]:
def tridiagPut(LL, DD, UU, B, x, K, NN):
    L = np.empty(NN+1)
    D = np.empty(NN+1)
    U = np.empty(NN+1)
    
    L[1:NN+1] = np.copy(LL[1:NN+1])
    D[1:NN+1] = np.copy(DD[1:NN+1])
    U[1:NN+1] = np.copy(UU[1:NN+1])
    
    for i in range(NN-1, 0, -1):
        Xmult = U[i] / D[i+1]
        D[i]  = D[i] - Xmult * L[i+1]
        B[i]  = B[i] - Xmult * B[i+1]
        
    Ic = 1
    B[Ic] = B[Ic] / D[Ic]
    while B[Ic] <= max(K - np.exp(x[Ic]), 0):
        B[Ic] = max(K - np.exp(x[Ic]), 0)
        B[Ic+1] = (B[Ic+1] - L[Ic+1] * B[Ic]) / D[Ic+1];
        Ic += 1
        
    index = Ic
    for i in range(Ic+1, NN+1):
        B[i] = (B[i] - L[i] * B[i-1]) / D[i];
    return index

In [76]:
def expint(x, verbose=True):
    
    # constants
    MAXIT = 100
    EULER = .577215664901532860606512
    FPMIN = 1.0e-30
    EPS = 1.0e-7
    
    n = 1
    nm1 = n-1
    
    if n < 0 or x < 0.0 or (x == 0 and (n == 0 or n == 1)):
        print('bad arguments in expint')
        ans = -1
    elif n == 0:
        ans = np.exp(-x) / x
    elif x == 0.0:
        ans = 1.0 / nm1
    elif x > 1.0:
        b = x + n
        c = 1.0 / FPMIN
        d = 1.0 / b
        h = d
        for i in range(1, MAXIT+1):
            a = -i*(nm1+i)
            b += 2.0
            d = 1.0/(a*d+b)
            c = b+a/c
            del_v = c*d
            h *= del_v
            if np.abs(del_v - 1.) < EPS:
                ans = h * np.exp(-x)
                return ans
        if verbose:
            print('continued fraction failed in expint')
            ans = -1
    else:
        if nm1 != 0:
            ans = 1.0 / nm1
        else:
            ans = -np.log(x) - EULER
        fact = 1.0
        for i in range(1, MAXIT):
            fact *= -1. * x / i
            if i != nm1:
                del_v = -1.* fact / (i - nm1)
            else:
                psi = -EULER
                for ii in range(1, nm1+1):
                    psi += 1.0 / ii
                del_v = fact * (-np.log(x) + psi)
            ans += del_v
            if (np.abs(del_v) < np.abs(ans)*EPS):
                return ans
        if verbose:
            print('series failed in expint')
    return ans

In [79]:
r = 0.0541
q = 0.012
sig = 0.20722
nu = 0.50215
theta = -0.22898
T = 0.56164
S0 = 1369.41
K = 1300.0
SMIN = 10
SMAX = 2000

# too slow!
N = 2000
M = 100

fout1 = open('prices.dat','w')
fout2 = open('freeboundary.dat','w')

Dx = (np.log(SMAX) - np.log(SMIN)) / N
Dt = 1. * T / M

L = np.empty(N)
D = np.empty(N)
U = np.empty(N)
B = np.empty(N)
W = np.empty(N+1)
x = np.empty(N+1)

s_tau = np.empty(M+1)

exp_n = np.empty(N)
exp_p = np.empty(N)

ei_n = np.empty(N)
ei_p = np.empty(N)

t = np.empty(M+1)

omega    = np.log(1-theta*nu-sig*sig*nu/2.)/nu
lambda_n = np.sqrt(theta * theta / sig**4 + 2 / (nu*sig*sig)) + theta/(sig*sig)
lambda_p = np.sqrt(theta * theta / sig**4 + 2 / (nu*sig*sig)) - theta/(sig*sig)

cn = Dt / (lambda_n * nu * Dx)
cp = Dt / (lambda_p * nu * Dx)

c = Dt / nu

Bn = cn*(1-np.exp(-lambda_n*Dx))
Bp = cp*(1-np.exp(-lambda_p*Dx))

A = (r-q+omega)*Dt/(2*Dx)

# Pre-calculated Vectors
for k in range(1, N):
    ei_n[k] = expint(k*Dx*lambda_n)
    ei_p[k] = expint(k*Dx*lambda_p)
    exp_n[k] = np.exp(-k*Dx*lambda_n)
    exp_p[k] = np.exp(-k*Dx*lambda_p)

counter = 0
for i in range(0, N+1):
    x[i] = np.log(SMIN) + i* Dx
    if np.exp(x[i]) < K:
        W[i] = K - np.exp(x[i])
    else:
        if counter == 0:
            index = i
            counter = 1
        W[i] = 0.0

s_tau[0] = K
t[0] = 0.0
for j in range(1, M+1):
    print('%i' % (M-j))
    t[j] = Dt*j
    # Boundary Conditions 
    W[0] = K - np.exp(x[0])
    W[N] = 0
    #
    B[1:N] = W[1:N]
    #
    for i in range(1, N):
        if i == 1:
            D[i] = 1 + r*Dt + Bp + Bn + c*(expint((N-i)*lambda_p*Dx)+expint(i*lambda_n*Dx))
            U[i] = -A - Bp
            for k in range(1,N-i-1+1):
                B[i] += c*((W[i+k]-W[i])-k*(W[i+k+1]-W[i+k]))*(ei_p[k]-ei_p[k+1]) + cp*(W[i+k+1]-W[i+k])*(exp_p[k]-exp_p[k+1])
            B[i] += c*( K*expint(i*Dx*lambda_n) - np.exp(x[i])*expint(i*Dx*(lambda_n+1)))
            B[i] += -(A - Bn)*( K - np.exp(x[0]) ) # Boundary Condition Effect    
        elif i == N-1:
            L[i] = A - Bn
            D[i] = 1 + r*Dt + Bp + Bn + c*(expint((N-i)*lambda_p*Dx)+expint(i*lambda_n*Dx))
            for k in range(1,i-1+1):
                B[i] += c*((W[i-k]-W[i])-k*(W[i-k-1]-W[i-k]))*(ei_n[k]-ei_n[k+1]) + cn*(W[i-k-1]-W[i-k])*(exp_n[k]-exp_n[k+1])
            B[i] += c*(K*expint(i*Dx*lambda_n) - np.exp(x[i])*expint(i*Dx*(lambda_n+1)))
        else:
            L[i] =  A - Bn
            D[i] =  1.0 + r*Dt + Bp + Bn + c*(expint((N-i)*lambda_p*Dx)+expint(i*lambda_n*Dx))
            U[i] = -A - Bp
            for k in range(1,N-i-1+1):
                B[i] += c*((W[i+k]-W[i])-k*(W[i+k+1]-W[i+k]))*(ei_p[k]-ei_p[k+1]) + cp*(W[i+k+1]-W[i+k])*(exp_p[k]-exp_p[k+1])
            for k in range(1,i-1+1):
                B[i] += c*((W[i-k]-W[i])-k*(W[i-k-1]-W[i-k]))*(ei_n[k]-ei_n[k+1]) + cn*(W[i-k-1]-W[i-k])*(exp_n[k]-exp_n[k+1])
            B[i] += c*(K*expint(i*Dx*lambda_n) - np.exp(x[i])*expint(i*Dx*(lambda_n+1)))
        
        if x[i] < x[index]:
            B[i] += Dt*(r*K - q*np.exp(x[i]))
            for k in range(index -i, N-i-1+1):
                B[i] -= c*((W[i+k]-W[i])-k*(W[i+k+1]-W[i+k]))*(ei_p[k]-ei_p[k+1]) + cp*(W[i+k+1]-W[i+k])*(exp_p[k]-exp_p[k+1])
            B[i] += c * ( K*expint((index-i)*Dx*lambda_p) - np.exp(x[i])*expint((index-i)*Dx*(lambda_p-1)) )
    
    W[1:N] =B[1:N]
    
    index = tridiagPut(L, D, U, W, x, K, N-1)
    #index = 0
    s_tau[j] = np.exp(x[index])

for i in range(0, N):
    if x[i] > np.log(S0):
        ir = i
        break

for i in range(0, N+1):
    fout1.write('%g %g \n' % (np.exp(x[i]), W[i]))

cs0 = (W[ir]-W[ir-1])*(np.log(S0)-x[ir-1])/Dx + W[ir-1]

print('\n Stock Price  ' + str(S0))
print('\n American Put Value  ' + str(cs0) + '\n')

for j in range(0, M+1):
    fout2.write('%g %g \n' % (t[j], s_tau[j]))
        
            
fout1.close()
fout2.close()

99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0

 Stock Price  1369.41

 American Put Value  58.27459025017218

